In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import os
import chromadb

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Initialize ChromaDB
chroma_client = chromadb.HttpClient(host='removed for security', port=0)
# collection = chroma_client.create_collection('documents')
try:
    collection = chroma_client.get_collection('documents')
except:
    collection = chroma_client.create_collection('documents')

In [ ]:
def process_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.readlines()
    embeddings = model.encode(text, show_progress_bar=True)
    i = 0
    for line, chunk in zip(text, embeddings):
        id = 'id' + str(i)
        collection.add(id, line, chunk.tolist())

    # collection.add(
    #     ids = filepath,
    #     documents = [text],
    #     embeddings = embeddings.tolist()
    # )

In [ ]:
files = []
for dirname, _, filenames in os.walk('linkedin_skills'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))

files

# for file in files:
#     process_file(file)

# process_file('linkedin_skills/small.txt')

In [ ]:
def retrieve_embed(query_text):
    # Generate embedding for the query
    query_embedding = model.encode([query_text])
    
    # Retrieve relevant documents
    results = collection.query(query_embedding.tolist(), n_results=5)
    return results

In [ ]:
embeds = retrieve_embed('Customer')
embeds

In [ ]:
embeds['documents'][0][0]